In [2]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://{}:{}@{}/{}?charset={}".format('root', '', 'localhost', 'final','utf8'))
con = engine.connect()#建立連線
sql = "DROP TABLE IF EXISTS sport,result"
con.execute(sql)

for i in range(1,31): #30天資料
    
    time='2022/04/' #時間的字串
    timenum=20220400
    web="https://www.playsport.cc/predictgame.php?action=result&allianceid=3&gametime=202204" #玩運彩的網址
    if i<10:
        web+="0"+str(i) #把web的網址拼完
        time+='0'+str(i) #更改time
    else:
        web+=str(i)
        time+=str(i)
    timenum=20220400+i    
    table=pd.read_html(web) #pandas讀網頁表格
    print(time)
    try:
        df=table[0] #轉成dataframe
        df_result=pd.DataFrame(columns=['讓分','不讓分','大小分','單雙'])#建result表
        index=-1; #index用來記同一場次
        df.columns=['賽事資訊','球隊資訊','國際讓分','國際大小','讓分','不讓分賠率','大小分','勝差'] #原始表格的column
        #增加需要的資料
        df.insert(5, "讓分賠率", "")
        df.insert(8, "大小分賠率", "")
        df.insert(10, "單雙賠率", "")
        df.insert(0 ,"日期", time)
        df.insert(1 ,"時間", "")
        df.insert(2 ,"編號", "")
        df.insert(3 ,"球隊", "")
        df.insert(4 ,"得分", "")
        df.insert(5 ,"總比數", "")
        #drop空欄
        for i in range(len(df)):
            if type(df.loc[i,'賽事資訊'])!=type('a'): #非字串(=空)
                df.drop([i],axis=0,inplace=True)
        df.reset_index(drop=True, inplace=True) #重設index

        #填資料
        for i in range(len(df)):
            #index
            if i%2!=1: index+=1
            df.loc[i,'編號']=str(timenum)+"-"+str(index)
            
            #時間
            l1=df.loc[i,'賽事資訊'].split()
            df.loc[i,'時間']=l1[1]+" "+l1[2]
            
            #球隊和比數
            l2=df.loc[i,'球隊資訊'].split()
            df.loc[i,'球隊']=l2[3] if i%2!=1 else l2[4]
            df.loc[i,'得分']=l2[0] if i%2!=1 else l2[2]
            df.loc[i,'總比數']=l2[0]+" : "+l2[2]
            
            #讓分
            l3=df.loc[i,'讓分'].split(', ')
            df.loc[i,'讓分']=l3[0][1:]
            bs=l3[0][0]
            df.loc[i,'讓分賠率']=bs+l3[1]

            #讓分過關
            if i%2!=1: #第一隊
                if int(l2[0])+float(l3[0][1:])>int(l2[2]):
                    df_result.loc[i,'讓分']=1
                else:
                    df_result.loc[i,'讓分']=0
            else:      #第二隊
                if int(l2[0])<int(l2[2])+float(l3[0][1:]):
                    df_result.loc[i,'讓分']=1
                else:
                    df_result.loc[i,'讓分']=0

            #不讓分        
            if type(df.loc[i,'不讓分賠率'])==type('a'):
                df.loc[i,'不讓分賠率']=df.loc[i,'不讓分賠率'][1:]

                #不讓分過關
                if i%2!=1:
                    if int(l2[0])>int(l2[2]):
                        df_result.loc[i,'不讓分']=1
                    else:
                        df_result.loc[i,'不讓分']=0 
                else:
                    if int(l2[0])<int(l2[2]):
                        df_result.loc[i,'不讓分']=1
                    else:
                        df_result.loc[i,'不讓分']=0
            else:
                df.loc[i,'不讓分賠率']=None
                df_result.loc[i,'不讓分']=None
            #大小分
            l4=df.loc[i,'大小分'].split(', ')
            df.loc[i,'大小分']=l4[0][1:]
            bs=l4[0][0]
            df.loc[i,'大小分賠率']=bs+l4[1]

            #大小分過關:1為大，0為小

            if int(l2[0])+int(l2[2])<float(l4[0][1:]):
                df_result.loc[i,'大小分']=0
            else:
                df_result.loc[i,'大小分']=1

            #單雙    
            df.loc[i,'單雙賠率']=('單' if i%2!=1 else "雙")+str(1.75)

            #單雙過關:1為單，0為雙
            if (int(l2[0])+int(l2[2]))%2:
                df_result.loc[i,'單雙']=1
            else:
                df_result.loc[i,'單雙']=0

        df.drop(['賽事資訊','球隊資訊','國際讓分','國際大小','勝差'],axis=1,inplace=True) #drop多餘資訊
        
        df.to_sql(name='sport', con=con, if_exists='append',index=False)
        df_result.to_sql(name='result', con=con, if_exists='append',index=False)
        
        #df.to_csv('sport.csv',encoding='big5',mode='a', index=False, header=False) #寫入csv
        #df_result.to_csv('result.csv',encoding='big5',mode='a', index=False, header=False) #寫入csv
        df_result = df_result.iloc[0:0]#清空result的dataframe
    except:
        print(time+" failed")
        continue


2022/04/01
2022/04/02
2022/04/03
2022/04/04
2022/04/05
2022/04/05 failed
2022/04/06
2022/04/07
2022/04/08
2022/04/09
2022/04/10
2022/04/11
2022/04/12
2022/04/12 failed
2022/04/13
2022/04/14
2022/04/15
2022/04/15 failed
2022/04/16
2022/04/17
2022/04/18
2022/04/19
2022/04/20
2022/04/21
2022/04/22
2022/04/23
2022/04/24
2022/04/25
2022/04/26
2022/04/27
2022/04/28
2022/04/29
2022/04/30
